In [1]:
from mcts import MCTS
from generator import Generator
from utils import Utils
from backtrack import Backtrack
from state import State

import time
import pandas as pd
import seaborn as sns

Generating puzzles

In [2]:
gen = Generator()
ps = [gen.getPuzzle(i) for i in range(5, 15) for _ in range(50)]
ps = {pid: p for pid, p in enumerate(ps)}
print(f"{len(ps)} puzzles to solve")

500 puzzles to solve


Creating a list of dictionaries which we'll turn into a dataframe later

In [3]:
data = []

Main loop of solving puzzles

In [4]:
for pid in ps:
    print(f"solving puzzle {pid} out of {len(ps)}")

    # MCTS, random
    mcts = MCTS(ps[pid])

    start = time.process_time()
    mcts.run()
    stop = time.process_time()
    
    t = stop - start
    vn = mcts.visited
    
    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"mcts",
        "heuristic":"false",
        "visited_nodes":vn,
        "time":t
    })

    # MCTS, Rank
    print(".")

    mcts = MCTS(ps[pid], h = "R")

    start = time.process_time()
    mcts.run()
    stop = time.process_time()
    
    t = stop - start
    vn = mcts.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"mcts",
        "heuristic":"true",
        "visited_nodes":vn,
        "time":t
    })

    # Backtrack, random
    print(".")
    bt = Backtrack(ps[pid])

    start = time.process_time()
    bt.run()
    stop = time.process_time()

    t = stop - start
    vn = bt.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"bt",
        "heuristic":"false",
        "visited_nodes":vn,
        "time":t
    })

    # Backtrack, Rank
    print(".")

    bt = Backtrack(ps[pid], h = "R")

    start = time.process_time()
    bt.run()
    stop = time.process_time()

    t = stop - start
    vn = bt.visited

    data.append({
        "pid":pid,
        "n":len(ps[pid].ps),
        "method":"bt",
        "heuristic":"true",
        "visited_nodes":vn,
        "time":t
    })

print("DONE")

solving puzzle 0 out of 500
.
.
.
solving puzzle 1 out of 500
.
.
.
solving puzzle 2 out of 500
.
.
.
solving puzzle 3 out of 500
.
.
.
solving puzzle 4 out of 500
.
.
.
solving puzzle 5 out of 500
.
.
.
solving puzzle 6 out of 500
.
.
.
solving puzzle 7 out of 500
.
.
.
solving puzzle 8 out of 500
.
.
.
solving puzzle 9 out of 500
.
.
.
solving puzzle 10 out of 500
.
.
.
solving puzzle 11 out of 500
.
.
.
solving puzzle 12 out of 500
.
.
.
solving puzzle 13 out of 500
.
.
.
solving puzzle 14 out of 500
.
.
.
solving puzzle 15 out of 500
.
.
.
solving puzzle 16 out of 500
.
.
.
solving puzzle 17 out of 500
.
.
.
solving puzzle 18 out of 500
.
.
.
solving puzzle 19 out of 500
.
.
.
solving puzzle 20 out of 500
.
.
.
solving puzzle 21 out of 500
.
.
.
solving puzzle 22 out of 500
.
.
.
solving puzzle 23 out of 500
.
.
.
solving puzzle 24 out of 500
.
.
.
solving puzzle 25 out of 500
.
.
.
solving puzzle 26 out of 500
.
.
.
solving puzzle 27 out of 500
.
.
.
solving puzzle 28 out of 500
.

KeyboardInterrupt: 

In [ ]:
data = pd.DataFrame(data)
data.to_csv("./data.csv")